<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/clustering_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf 'science_science'
username = "sahandv"#@param {type:"string"}
# password = ""#@param {type:"string"} 
!git clone https://github.com/$username/science_science.git
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!pip install -r 'science_science/requirements.txt'
!pip install tensorflow
!pip install keras

In [1]:
# datapath = '/mnt/6016589416586D52/Users/z5204044/GoogleDrive/GoogleDrive/Data/' # Local C1314
datapath = '/mnt/16A4A9BCA4A99EAD/GoogleDrive/Data/' # Local Ryzen
# datapath = 'drive/My Drive/Data/' # Remote

In [2]:
# Remote
# from science_science.sciosci.assets import text_assets as ta
# from science_science.DEC.DEC_keras import DEC_simple_run

# Local
from sciosci.assets import text_assets as ta
from DEC.DEC_keras import DEC_simple_run

In [4]:
import sys
import time
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from random import randint

from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn import metrics
from sklearn.metrics.cluster import silhouette_score,homogeneity_score,adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score,adjusted_mutual_info_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer , TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing


# =============================================================================
# Load data and init
# =============================================================================
data_address =  datapath+"Corpus/KPRIS/embeddings/deflemm/Doc2Vec patent_wos corpus"
label_address =  datapath+"Corpus/KPRIS/labels"

vectors = pd.read_csv(data_address)
labels = pd.read_csv(label_address,names=['label'])
labels_f = pd.factorize(labels.label)
X = vectors.values
Y = labels_f[0]
n_clusters = 5

labels_task_1 = labels[(labels['label']=='car') | (labels['label']=='memory')]
vectors_task_1 = vectors.iloc[labels_task_1.index]
labels_task_1_f = pd.factorize(labels_task_1.label)
X_task_1 = vectors_task_1.values
Y_task_1 = labels_task_1_f[0]
n_clusters_task_1 = 2

results = pd.DataFrame([],columns=['Method','parameter','Silhouette','Homogeneity','NMI','AMI','ARI'])
# =============================================================================
# Evaluation method
# =============================================================================
def evaluate(X,Y,predicted_labels):
    
    df = pd.DataFrame(predicted_labels,columns=['label'])
    if len(df.groupby('label').groups)<2:
        return [0,0,0,0,0]
    
    return [silhouette_score(X, predicted_labels, metric='euclidean'),
                    homogeneity_score(Y, predicted_labels),
                    normalized_mutual_info_score(Y, predicted_labels),
                    adjusted_mutual_info_score(Y, predicted_labels),
                    adjusted_rand_score(Y, predicted_labels)]

In [ ]:

# =============================================================================
# K-means
# =============================================================================
print('\n- k-means random -----------------------')
for fold in tqdm(range(20)):
    seed = randint(0,10**5)
    model = KMeans(n_clusters=n_clusters,n_init=20, init='random', random_state=seed).fit(X)
    predicted_labels = model.labels_
    tmp_results = ['k-means random','seed '+str(seed)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# K-means with init='k-means++'
# =============================================================================
print('\n- k-means++ -----------------------')
for fold in tqdm(range(20)):
    seed = randint(0,10**5)
    model = KMeans(n_clusters=n_clusters,n_init=20,init='k-means++', random_state=seed).fit(X)
    predicted_labels = model.labels_
    tmp_results = ['k-means++','seed '+str(seed)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Agglomerative
# =============================================================================
print('\n- Agglomerative -----------------------')
for fold in tqdm(range(4)):
    model = AgglomerativeClustering(n_clusters=n_clusters,linkage='ward').fit(X)
    predicted_labels = model.labels_
    tmp_results = ['Agglomerative','ward']+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# DBSCAN
# =============================================================================
eps=0.000001
print('\n- DBSCAN -----------------------')
for fold in tqdm(range(19)):
    eps = eps+0.05
    model = DBSCAN(eps=eps, min_samples=10,n_jobs=15).fit(X)
    predicted_labels = model.labels_
    tmp_results = ['DBSCAN','eps '+str(eps)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Deep no min_max_scaling
# =============================================================================
archs = [[500, 500, 2000, 10],[500, 1000, 2000, 10],[500, 1000, 1000, 10],
         [500, 500, 2000, 100],[500, 1000, 2000, 100],[500, 1000, 1000, 100],
         [100, 300, 600, 10],[300, 500, 2000, 10],[700, 1000, 2000, 10],
         [200, 500, 10],[500, 1000, 10],[1000, 2000, 10],
         [200, 500, 100],[500, 1000, 100],[1000, 2000, 100],
         [1000, 500, 10],[500, 200, 10],[200, 100, 10],
         [1000, 1000, 2000, 10],[1000, 1500, 2000, 10],[1000, 1500, 1000, 10],
         [1000, 1000, 2000,500, 10],[1000, 1500, 2000,500, 10],[1000, 1500, 1000, 500, 10],
         [500, 500, 2000, 500, 10],[500, 1000, 2000, 500, 10],[500, 1000, 1000, 500, 10]]
print('\n- DEC -----------------------')
for fold in tqdm(archs):
    predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=False,n_clusters=5,architecture=fold,pretrain_epochs=300)
    tmp_results = ['DEC',str(fold)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Deep with min_max_scaling
# =============================================================================
archs = [[500, 500, 2000, 10],[500, 1000, 2000, 10],[500, 1000, 1000, 10],
         [500, 500, 2000, 100],[500, 1000, 2000, 100],[500, 1000, 1000, 100],
         [100, 300, 600, 10],[300, 500, 2000, 10],[700, 1000, 2000, 10],
         [200, 500, 10],[500, 1000, 10],[1000, 2000, 10],
         [200, 500, 100],[500, 1000, 100],[1000, 2000, 100],
         [1000, 500, 10],[500, 200, 10],[200, 100, 10],
         [1000, 1000, 2000, 10],[1000, 1500, 2000, 10],[1000, 1500, 1000, 10],
         [1000, 1000, 2000,500, 10],[1000, 1500, 2000,500, 10],[1000, 1500, 1000, 500, 10],
         [500, 500, 2000, 500, 10],[500, 1000, 2000, 500, 10],[500, 1000, 1000, 500, 10]]
print('\n- DEC -----------------------')
for fold in tqdm(archs):
    predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=True,n_clusters=5,architecture=fold,pretrain_epochs=300)
    tmp_results = ['DEC minmax scaler',str(fold)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)


In [ ]:
# =============================================================================
# Save to disk
# =============================================================================
results_df = pd.DataFrame(results)
results_df.to_csv(data_address+' clustering results _ new',index=False)

Mini test

In [8]:
# =============================================================================
# Deep no min_max_scaling
# =============================================================================
archs = [[500, 1000,10],[200,500,10],[200, 200,10],
        [200,200,10],[200,400,10],[200,600,10],
        [200,1000,10],[500,1000,10,5],[500,1000,5],
        [200,200,5],[200,400,5],[200,600,5]]
archs = [[200,200,10],[200,200,10],[200,200,10],
        [200,200,10],[200,200,10],[200,200,10],
        [200,200,10],[200,200,10],[200,200,10],
        [200,500,10],[200,500,10],[200,500,10],
        [200,500,10],[200,500,10],[200,500,10]]

print('\n- DEC -----------------------')
for fold in tqdm(archs):
    seed = randint(0,10**4)
    np.random.seed()
    predicted_labels = DEC_simple_run(X,minmax_scale_custom_data=False,n_clusters=5,architecture=fold,pretrain_epochs=300)
    tmp_results = ['DEC',str(seed)+' '+str(fold)]+evaluate(X,Y,predicted_labels)
    tmp_results = pd.Series(tmp_results, index = results.columns)
    results = results.append(tmp_results, ignore_index=True)
mean = results.mean(axis=0)
maxx = results.max(axis=0)
print(mean)
print(maxx)
# =============================================================================
# Save to disk
# =============================================================================
results_df = pd.DataFrame(results)
results_df.to_csv(data_address+' clustering results _ new 3',index=False)

  0%|          | 0/15 [00:00<?, ?it/s]


- DEC -----------------------
Model: "AE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 100)]             0         
_________________________________________________________________
encoder_0 (Dense)            (None, 200)               20200     
_________________________________________________________________
encoder_1 (Dense)            (None, 200)               40200     
_________________________________________________________________
encoder_2 (Dense)            (None, 10)                2010      
_________________________________________________________________
decoder_2 (Dense)            (None, 200)               2200      
_________________________________________________________________
decoder_1 (Dense)            (None, 200)               40200     
_________________________________________________________________
decoder_0 (Dense)            (Non

77/77 [==============================] - 0s 2ms/step - loss: 4.6097
Epoch 85/300
77/77 [==============================] - 0s 1ms/step - loss: 4.6088
Epoch 86/300
77/77 [==============================] - 0s 2ms/step - loss: 4.6055
Epoch 87/300
77/77 [==============================] - 0s 2ms/step - loss: 4.6049
Epoch 88/300
77/77 [==============================] - 0s 2ms/step - loss: 4.6007
Epoch 89/300
77/77 [==============================] - 0s 2ms/step - loss: 4.5998
Epoch 90/300
77/77 [==============================] - 0s 2ms/step - loss: 4.5981
Epoch 91/300
77/77 [==============================] - 0s 2ms/step - loss: 4.5946
Epoch 92/300
77/77 [==============================] - 0s 2ms/step - loss: 4.5911
Epoch 93/300
77/77 [==============================] - 0s 1ms/step - loss: 4.5893
Epoch 94/300
77/77 [==============================] - 0s 2ms/step - loss: 4.5873
Epoch 95/300
77/77 [==============================] - 0s 2ms/step - loss: 4.5850
Epoch 96/300
77/77 [=====================

77/77 [==============================] - 0s 2ms/step - loss: 4.4627
Epoch 185/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4598
Epoch 186/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4603
Epoch 187/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4579
Epoch 188/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4584
Epoch 189/300
77/77 [==============================] - 0s 1ms/step - loss: 4.4580
Epoch 190/300
77/77 [==============================] - 0s 1ms/step - loss: 4.4559
Epoch 191/300
77/77 [==============================] - 0s 1ms/step - loss: 4.4536
Epoch 192/300
77/77 [==============================] - 0s 1ms/step - loss: 4.4544
Epoch 193/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4529
Epoch 194/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4517
Epoch 195/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4513
Epoch 196/300
77/77 [=========

77/77 [==============================] - 0s 1ms/step - loss: 4.3939
Epoch 285/300
77/77 [==============================] - 0s 2ms/step - loss: 4.3926
Epoch 286/300
77/77 [==============================] - 0s 1ms/step - loss: 4.3929
Epoch 287/300
77/77 [==============================] - 0s 2ms/step - loss: 4.3920
Epoch 288/300
77/77 [==============================] - 0s 2ms/step - loss: 4.3945
Epoch 289/300
77/77 [==============================] - 0s 1ms/step - loss: 4.3908
Epoch 290/300
77/77 [==============================] - 0s 2ms/step - loss: 4.3911
Epoch 291/300
77/77 [==============================] - 0s 1ms/step - loss: 4.3918
Epoch 292/300
77/77 [==============================] - 0s 1ms/step - loss: 4.3914
Epoch 293/300
77/77 [==============================] - 0s 1ms/step - loss: 4.3947
Epoch 294/300
77/77 [==============================] - 0s 2ms/step - loss: 4.3903
Epoch 295/300
77/77 [==============================] - 0s 2ms/step - loss: 4.3888
Epoch 296/300
77/77 [=========

...

77/77 [==============================] - 0s 2ms/step - loss: 4.4639
Epoch 86/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4598
Epoch 87/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4567
Epoch 88/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4524
Epoch 89/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4490
Epoch 90/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4436
Epoch 91/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4399
Epoch 92/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4367
Epoch 93/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4304
Epoch 94/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4280
Epoch 95/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4267
Epoch 96/300
77/77 [==============================] - 0s 2ms/step - loss: 4.4244
Epoch 97/300
77/77 [=====================

77/77 [==============================] - 0s 2ms/step - loss: 4.2163
Epoch 186/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2151
Epoch 187/300
77/77 [==============================] - 0s 1ms/step - loss: 4.2109
Epoch 188/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2091
Epoch 189/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2093
Epoch 190/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2094
Epoch 191/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2095
Epoch 192/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2083
Epoch 193/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2066
Epoch 194/300
77/77 [==============================] - 0s 1ms/step - loss: 4.2030
Epoch 195/300
77/77 [==============================] - 0s 1ms/step - loss: 4.2017
Epoch 196/300
77/77 [==============================] - 0s 2ms/step - loss: 4.2034
Epoch 197/300
77/77 [=========

77/77 [==============================] - 0s 2ms/step - loss: 4.1046
Epoch 286/300
77/77 [==============================] - 0s 2ms/step - loss: 4.1020
Epoch 287/300
77/77 [==============================] - 0s 2ms/step - loss: 4.1020
Epoch 288/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0990
Epoch 289/300
77/77 [==============================] - 0s 2ms/step - loss: 4.1023
Epoch 290/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0985
Epoch 291/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0949
Epoch 292/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0947
Epoch 293/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0956
Epoch 294/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0917
Epoch 295/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0955
Epoch 296/300
77/77 [==============================] - 0s 2ms/step - loss: 4.0926
Epoch 297/300
77/77 [=========

100%|██████████| 15/15 [24:17<00:00, 97.17s/it]

Silhouette     0.025653
Homogeneity    0.683161
NMI            0.677449
AMI            0.677365
ARI            0.696617
dtype: float64
Method                     DEC
parameter      [500, 1000, 10]
Silhouette           0.0330702
Homogeneity           0.740582
NMI                   0.733854
AMI                   0.733785
ARI                   0.758855
dtype: object
